In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1"

In [2]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import matrix_product_state
from tensorly import tt_to_tensor
from scipy.sparse import csr_matrix
import torch
from multiprocessing import Pool
import scipy
from tensorly.tenalg import contract
import pickle
import itertools
import scipy
import tqdm

In [3]:
def tensordot(A,B,modes):
    return contract(A,modes[0],B,modes[1])

In [4]:
def TTOI(Y_tensor, r_vec, iter, tol):
    dim_vec = Y_tensor.shape
    d = len(dim_vec)
    X_hat_arr = [None] * iter
    V_prod_arr = [None] * int(np.floor(iter/2)+1)
    U_prod_arr = [None] * int(np.ceil(iter/2))
    Y_arr = [None] * (d-1)
    for i in range(d-1):
        Y_arr[i] = Y_tensor.reshape(np.prod(dim_vec[:i+1]), np.prod(dim_vec[i+1:]), order="F")
    V_prod_arr[0] = [1] * (d-1)
    n = 0
    chg = np.Inf
    while n<iter and chg > tol:
        if n==0:
            U_prod_arr[int(n / 2)] = [None] * (d - 1)
            Y_tilde_arr = [None] * (d - 2)
            U_temp, _, _ = np.linalg.svd(Y_arr[0])
            U_temp = U_temp[:, :r_vec[0]]
            U_prod_arr[int(n / 2)][0] = U_temp
            for k in range(1, d - 1):
                Y_temp = np.kron(np.eye(dim_vec[k]), U_prod_arr[int(n / 2)][k - 1]).T @ Y_arr[k]
                Y_tilde_arr[k - 1] = Y_temp.reshape(r_vec[k - 1], np.prod(dim_vec[k:]), order="F")
                U_temp, _, _ = np.linalg.svd(Y_temp)
                U_temp = U_temp[:, :r_vec[k]]
                U_prod_arr[int(n / 2)][k] = np.kron(np.eye(dim_vec[k]), U_prod_arr[int(n / 2)][k - 1]) @ U_temp
            X_hat_temp = U_prod_arr[int(n / 2)][d - 2].T @ Y_arr[d - 2]
            X_hat_arr[n] = (U_prod_arr[int(n / 2)][d - 2] @ X_hat_temp).reshape(dim_vec, order="F")
        elif n % 2 == 0:
            U_prod_arr[int(n/2)] = [None] * (d-1)
            Y_tilde_arr = [None] * (d-2)
            U_temp, _, _ = np.linalg.svd(Y_arr[0]@V_prod_arr[int(n/2)][d-2])
            U_temp = U_temp[:,:r_vec[0]]
            U_prod_arr[int(n/2)][0] = U_temp
            for k in range(1,d-1):
                Y_temp = np.kron(np.eye(dim_vec[k]),U_prod_arr[int(n/2)][k-1]).T@Y_arr[k]
                Y_tilde_arr[k-1] = Y_temp.reshape(r_vec[k-1],np.prod(dim_vec[k:]),order="F")
                U_temp, _, _ = np.linalg.svd(Y_temp@V_prod_arr[int(n/2)][d-k-2])
                U_temp = U_temp[:,:r_vec[k]]
                U_prod_arr[int(n/2)][k] = np.kron(np.eye(dim_vec[k]),U_prod_arr[int(n/2)][k-1])@U_temp
            X_hat_temp = U_prod_arr[int(n/2)][d-2].T@Y_arr[d-2]
            X_hat_arr[n] = (U_prod_arr[int(n/2)][d-2]@X_hat_temp).reshape(dim_vec,order="F")
        else:
            V_prod_arr[int((n+1)/2)] = [None] * (d-1)
            _, _, V_temp = np.linalg.svd(U_prod_arr[int((n-1)/2)][d-2].T@Y_arr[d-2])
            V_temp = V_temp[:r_vec[d-2],:].T
            V_prod_arr[int((n+1)/2)][0] = V_temp
            for k in range(1,d-1):
                _, _, V_temp = np.linalg.svd(Y_tilde_arr[d-k-2]@np.kron(V_prod_arr[int((n+1)/2)][k-1],np.eye(dim_vec[d-k-1])))
                V_temp = V_temp[:r_vec[d - k-2], :].T
                V_prod_arr[int((n+1)/2)][k] = np.kron(V_prod_arr[int((n+1)/2)][k-1],np.eye(dim_vec[d-k-1]))@V_temp
            X_hat_temp = Y_arr[0]@V_prod_arr[int((n+1)/2)][d-2]
            X_hat_arr[n] = (X_hat_temp@V_prod_arr[int((n+1)/2)][d-2].T).reshape(dim_vec,order="F")
        n=n+1
        if n>1:
            chg = np.square(np.linalg.norm(X_hat_arr[n-1].reshape(np.prod(dim_vec),1),"fro"))-np.square(np.linalg.norm(X_hat_arr[n-2].reshape(np.prod(dim_vec),1),"fro"))

    return [x for x in X_hat_arr if x is not None]

In [5]:
def RGrad5(X, ranks, A):
    p1, p2, p3, p4, p5 = X.shape
    _, r1, r2, r3, r4, _ = ranks
    factors = matrix_product_state(X, rank=ranks)
    G1, G2, G3, G4, G5 = factors
    G1 = G1.reshape(p1,r1,order="F")
    G5 = G5.reshape(r4,p5,order="F")

    G_ge2 = (tensordot(tensordot(tensordot(G2,G3,[2,0]),G4,[3,0]),G5,[4,0])).reshape(r1, p2*p3*p4*p5, order="F")
    A1 = (np.identity(p1)-G1@G1.T)@A.reshape(p1,p2*p3*p4*p5,order="F")@G_ge2.T@np.linalg.inv(G_ge2@G_ge2.T)
    delta_A1 = tensordot(tensordot(tensordot(tensordot(A1,G2,[1,0]),G3,[2,0]),G4,[3,0]),G5,[4,0])

    G_ge3 = (tensordot(tensordot(G3,G4,[2,0]),G5,[3,0])).reshape(r2, p3*p4*p5, order="F")
    L_G2 = G2.reshape(p2*r1, r2, order="F")
    A2 = (np.identity(p2*r1)-L_G2@L_G2.T)@(np.kron(np.identity(p2), G1)).T@A.reshape(p1*p2,p3*p4*p5,order="F")@G_ge3.T@np.linalg.inv(G_ge3@G_ge3.T)
    delta_A2 = tensordot(tensordot(tensordot(tensordot(G1, A2.reshape([r1,p2,r2],order="F"), [1, 0]), G3, [2, 0]), G4, [3, 0]), G5, [4,0])

    G_ge4 = (tensordot(G4,G5,[2,0])).reshape(r3, p4*p5, order="F")
    L_G3 = G3.reshape(p3*r2, r3, order="F")
    G_le2 = (tensordot(G1,G2,[1,0])).reshape(p1*p2,r2, order="F")
    A3 = (np.identity(p3*r2)-L_G3@L_G3.T)@(np.kron(np.identity(p3), G_le2)).T@A.reshape(p1*p2*p3,p4*p5,order="F")@G_ge4.T@np.linalg.inv(G_ge4@G_ge4.T)
    delta_A3 = tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), A3.reshape([r2,p3,r3],order="F"), [2, 0]), G4, [3, 0]), G5, [4,0])

    L_G4 = G4.reshape(p4*r3, r4, order="F")
    G_le3 = (tensordot(tensordot(G1,G2,[1,0]),G3,[2,0])).reshape(p1*p2*p3,r3, order="F")
    A4 = (np.identity(p4*r3)-L_G4@L_G4.T)@(np.kron(np.identity(p4),G_le3)).T@A.reshape(p1*p2*p3*p4,p5,order="F")@G5.T @np.linalg.inv(G5@G5.T)
    delta_A4 = tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), A4.reshape([r3,p4,r4], order="F"), [3, 0]),G5,[4,0])

    G_le4 = (tensordot(tensordot(tensordot(G1,G2,[1,0]),G3,[2,0]),G4,[3,0])).reshape(p1*p2*p3*p4,r4,order="F")
    A5 = (np.kron(np.identity(p5), G_le4)).T@A.reshape(p1*p2*p3*p4*p5,1,order="F")
    delta_A5 = tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), G4,[3,0]),A5.reshape(r4,p5,order="F"), [4, 0])

    return delta_A1+delta_A2+delta_A3+delta_A4+delta_A5

In [6]:
import time
def PGD(Y, X, trueA, p_vec1, p_vec, mtl_rank, max_iter, tol):
    p1, N = Y.shape
    p, _ = X.shape
    A_inittmp = (Y@X.T)/N
    A_initts = TTOI(A_inittmp.reshape(p_vec1+p_vec,order="F"),mtl_rank[1:-1],2,1e-4)[-1]
    Amat = A_initts.reshape(p1,p,order="F")
    chg_list = [None] * max_iter
    est_list = [None] * max_iter
    n = 0
    chg = np.inf
    while n < max_iter and chg > tol:
        start = time.time()
        P = RGrad5(Amat.reshape(p_vec1+p_vec,order="F"),mtl_rank,((Amat@X-Y)@X.T)/N)
        Pmat = P.reshape(p1,p,order="F")
        eta = N*np.square(np.linalg.norm(Pmat,"fro"))/np.square(np.linalg.norm(Pmat@X,"fro"))
        A_tmpmat = Amat - eta*Pmat
        factors = matrix_product_state(A_tmpmat.reshape(p_vec1+p_vec,order="F"), rank=mtl_rank)
        A_newts = tt_to_tensor(factors)
        A_newmat = A_newts.reshape(p1,p,order="F")
        chg_list[n] = np.linalg.norm(A_newmat-Amat, "fro")
        est_list[n] = np.linalg.norm(A_newmat-trueA, "fro")
        chg = np.linalg.norm(A_newmat-Amat, "fro")
        if chg>10:
            break
        n=n+1
        Amat = A_newmat
        

    return ([x for x in chg_list if x is not None],[x for x in est_list if x is not None],Amat)

In [7]:
with open('33jointr2.pkl','rb') as f:
    A_mat, A_ts= pickle.load(f)

In [8]:
lambda1=np.linalg.svd(A_ts.reshape(6,36*36,order="F"))[1][:2]
lambda2=np.linalg.svd(A_ts.reshape(36,6*36,order="F"))[1][:2]
lambda3=np.linalg.svd(A_ts.reshape(216,36,order="F"))[1][:2]
lambda4=np.linalg.svd(A_ts.reshape(216*6,6,order="F"))[1][:2]
lambdatotal=np.concatenate((lambda1,lambda2,lambda3,lambda4))
np.min(lambdatotal)

2.7469435305966434

In [9]:
def d1(ind):
    torch.random.seed()
    N=200
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    a = list(range(1,4))
    b = list(range(1,4))
    c = list(range(1,4))
    d = list(range(1,4))
    global paramlist;
    paramlist = list(itertools.product(a,b,c,d))
    
    result1 = []
    t1 = 0
    t2 = 0
    for i in paramlist:
        result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[i[0]]+[i[1]]+[i[2]]+[i[3]]+[1], max_iter=2000, tol=1e-4)
        result1.append(result)
        if result[0][-1]>0.01:
            t1 = t1 + 1
        if len(result[1])>1998:
            t2 = t2 + 1
            
    rBIC = []
    rsigma = []
    for i in range(len(result1)):
        sigma = (np.linalg.norm(Y-result1[i][2]@X)**2)/(Y.shape[1])
        a,b,c,d = paramlist[i]
        df = 6*(a+d)+6*(a*b+b*c+c*d)+b
        BIC = np.log(sigma)+0.02*np.log(Y.shape[1])*(df+1)/(Y.shape[1])
        rBIC.append(BIC)
        rsigma.append(np.log(sigma))

    return (rBIC,paramlist[np.argsort(np.array(rBIC))[0]],t1,t2,rsigma,Y,X)


In [10]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(40) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=500))
        with open('N200.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

100%|██████████| 500/500 [37:34<00:00,  4.51s/it]  


2256.239568710327


In [17]:
def d2(ind):
    torch.random.seed()
    N=165
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    a = list(range(1,4))
    b = list(range(1,4))
    c = list(range(1,4))
    d = list(range(1,4))
    global paramlist;
    paramlist = list(itertools.product(a,b,c,d))
    
    result1 = []
    t1 = 0
    t2 = 0
    for i in paramlist:
        result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[i[0]]+[i[1]]+[i[2]]+[i[3]]+[1], max_iter=2000, tol=1e-4)
        result1.append(result)
        if result[0][-1]>0.01:
            t1 = t1 + 1
        if len(result[1])>1998:
            t2 = t2 + 1
            
    rBIC = []
    rsigma = []
    for i in range(len(result1)):
        sigma = (np.linalg.norm(Y-result1[i][2]@X)**2)/(Y.shape[1])
        a,b,c,d = paramlist[i]
        df = 6*(a+d)+6*(a*b+b*c+c*d)+b
        BIC = np.log(sigma)+0.02*np.log(Y.shape[1])*(df+1)/(Y.shape[1])
        rBIC.append(BIC)
        rsigma.append(np.log(sigma))

    return (rBIC,paramlist[np.argsort(np.array(rBIC))[0]],t1,t2,rsigma,Y,X)


In [18]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(40) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=500))
        with open('N165.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

100%|██████████| 500/500 [45:19<00:00,  5.44s/it]   


2723.2666358947754


In [19]:
def d3(ind):
    torch.random.seed()
    N=130
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    a = list(range(1,4))
    b = list(range(1,4))
    c = list(range(1,4))
    d = list(range(1,4))
    global paramlist;
    paramlist = list(itertools.product(a,b,c,d))
    
    result1 = []
    t1 = 0
    t2 = 0
    for i in paramlist:
        result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[i[0]]+[i[1]]+[i[2]]+[i[3]]+[1], max_iter=2000, tol=1e-4)
        result1.append(result)
        if result[0][-1]>0.01:
            t1 = t1 + 1
        if len(result[1])>1998:
            t2 = t2 + 1
            
    rBIC = []
    rsigma = []
    for i in range(len(result1)):
        sigma = (np.linalg.norm(Y-result1[i][2]@X)**2)/(Y.shape[1])
        a,b,c,d = paramlist[i]
        df = 6*(a+d)+6*(a*b+b*c+c*d)+b
        BIC = np.log(sigma)+0.02*np.log(Y.shape[1])*(df+1)/(Y.shape[1])
        rBIC.append(BIC)
        rsigma.append(np.log(sigma))

    return (rBIC,paramlist[np.argsort(np.array(rBIC))[0]],t1,t2,rsigma,Y,X)


In [20]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(40) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=500))
        with open('N130.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

100%|██████████| 500/500 [53:29<00:00,  6.42s/it]  


3212.967806339264


In [21]:
def d4(ind):
    torch.random.seed()
    N=95
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    a = list(range(1,4))
    b = list(range(1,4))
    c = list(range(1,4))
    d = list(range(1,4))
    global paramlist;
    paramlist = list(itertools.product(a,b,c,d))
    
    result1 = []
    t1 = 0
    t2 = 0
    for i in paramlist:
        result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[i[0]]+[i[1]]+[i[2]]+[i[3]]+[1], max_iter=2000, tol=1e-4)
        result1.append(result)
        if result[0][-1]>0.01:
            t1 = t1 + 1
        if len(result[1])>1998:
            t2 = t2 + 1
            
    rBIC = []
    rsigma = []
    for i in range(len(result1)):
        sigma = (np.linalg.norm(Y-result1[i][2]@X)**2)/(Y.shape[1])
        a,b,c,d = paramlist[i]
        df = 6*(a+d)+6*(a*b+b*c+c*d)+b
        BIC = np.log(sigma)+0.02*np.log(Y.shape[1])*(df+1)/(Y.shape[1])
        rBIC.append(BIC)
        rsigma.append(np.log(sigma))

    return (rBIC,paramlist[np.argsort(np.array(rBIC))[0]],t1,t2,rsigma,Y,X)


In [22]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(40) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=500))
        with open('N95.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

100%|██████████| 500/500 [1:13:47<00:00,  8.85s/it] 


4429.459269523621


In [23]:
def d5(ind):
    torch.random.seed()
    N=60
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    a = list(range(1,4))
    b = list(range(1,4))
    c = list(range(1,4))
    d = list(range(1,4))
    global paramlist;
    paramlist = list(itertools.product(a,b,c,d))
    
    result1 = []
    t1 = 0
    t2 = 0
    for i in paramlist:
        result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[i[0]]+[i[1]]+[i[2]]+[i[3]]+[1], max_iter=2000, tol=1e-4)
        result1.append(result)
        if result[0][-1]>0.01:
            t1 = t1 + 1
        if len(result[1])>1998:
            t2 = t2 + 1
            
    rBIC = []
    rsigma = []
    for i in range(len(result1)):
        sigma = (np.linalg.norm(Y-result1[i][2]@X)**2)/(Y.shape[1])
        a,b,c,d = paramlist[i]
        df = 6*(a+d)+6*(a*b+b*c+c*d)+b
        BIC = np.log(sigma)+0.02*np.log(Y.shape[1])*(df+1)/(Y.shape[1])
        rBIC.append(BIC)
        rsigma.append(np.log(sigma))

    return (rBIC,paramlist[np.argsort(np.array(rBIC))[0]],t1,t2,rsigma,Y,X)


In [24]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(40) as p:
        result = list(tqdm.tqdm(p.imap(d5,l),total=500))
        with open('N60.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

100%|██████████| 500/500 [2:45:35<00:00, 19.87s/it]    


9938.091502428055


In [25]:
def d6(ind):
    torch.random.seed()
    N=25
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    a = list(range(1,4))
    b = list(range(1,4))
    c = list(range(1,4))
    d = list(range(1,4))
    global paramlist;
    paramlist = list(itertools.product(a,b,c,d))
    
    result1 = []
    t1 = 0
    t2 = 0
    for i in paramlist:
        result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[i[0]]+[i[1]]+[i[2]]+[i[3]]+[1], max_iter=2000, tol=1e-4)
        result1.append(result)
        if result[0][-1]>0.01:
            t1 = t1 + 1
        if len(result[1])>1998:
            t2 = t2 + 1
            
    rBIC = []
    rsigma = []
    for i in range(len(result1)):
        sigma = (np.linalg.norm(Y-result1[i][2]@X)**2)/(Y.shape[1])
        a,b,c,d = paramlist[i]
        df = 6*(a+d)+6*(a*b+b*c+c*d)+b
        BIC = np.log(sigma)+0.02*np.log(Y.shape[1])*(df+1)/(Y.shape[1])
        rBIC.append(BIC)
        rsigma.append(np.log(sigma))

    return (rBIC,paramlist[np.argsort(np.array(rBIC))[0]],t1,t2,rsigma,Y,X)


In [26]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(40) as p:
        result = list(tqdm.tqdm(p.imap(d6,l),total=500))
        with open('N25.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

100%|██████████| 500/500 [5:22:45<00:00, 38.73s/it]    


19368.279304265976


In [13]:
with open('N200.pkl','rb') as f:
    result = pickle.load(f)
list1 = []
a1 = 0
t1 = []
a2 = 0
t2 = []
a3 = 0
total = 0
for i in range(len(result)):
    total+=1
    list1.append(result[i][1])
    if list(result[i][1])==[2,2,2,2]:
        a1 = a1 + 1
    t1.append(result[i][2])
    if result[i][2]>0:
        a2 = a2 + 1
    t2.append(result[i][3])
    if result[i][3]>0:
        a3 = a3 + 1

In [14]:
print(a1)
print(total)

500
500


In [14]:
list1

[(2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2,

In [ ]:
def d2(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    T=150
    p=125
    torch.random.seed()       
        
    #import time
    #start = time.time()
    global ts; global X; global Y;
    #ts = torch.empty((p, T), dtype=torch.float64)
    X = torch.empty((p, T), dtype=torch.float64)
    Y = torch.empty((p, T), dtype=torch.float64)
    X[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    #ts[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    for s in range(0,T):
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = X.numpy()
        Y = Y.numpy()
        if s!=T-1:
            X[:,s+1] = (np.transpose((Y[:,s].reshape(list(reversed(p_vec)),order="F")),[2,1,0])).reshape(p,order="F")
        #ts[:,s] = torch.tensor(A_mat) @ ts[:,s-1] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        #print(s)
    #end = time.time()
    #print(end-start)
    #ts = ts.nump()
    X = X.numpy()
    Y = Y.numpy()
    
    #a = list(range(1,4))
    #b = list(range(1,4))
    #c = list(range(1,4))
    #d = list(range(1,4))
    #e = list(range(1,4))
    #global paramlist;
    #paramlist = list(itertools.product(a,b,c,d,e))
    
    rmax = 3
    result1 = []
    result2 = []
    result3 = []
    result4 = []
    result5 = []
    t1 = 0
    t2 = 0
    for i in range(1,4):
        result1a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[i]+[rmax,rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result1.append(result1a)
        if result1a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result1a[1])>1998:
            t2 = t2 + 1
        
        result2a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax]+[i]+[rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result2.append(result2a)
        if result2a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result2a[1])>1998:
            t2 = t2 + 1
        
        result3a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax]+[i]+[rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result3.append(result3a)
        if result3a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result3a[1])>1998:
            t2 = t2 + 1
        
        result4a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax]+[i]+[rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result4.append(result4a)
        if result4a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result4a[1])>1998:
            t2 = t2 + 1
        
        result5a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax,rmax]+[i], eta=0.01, max_iter=2000, tol=1e-4)
        result5.append(result5a)
        if result5a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result5a[1])>1998:
            t2 = t2 + 1
        
        
    rBIC1 = []
    rBIC2 = []
    rBIC3 = []
    rBIC4 = []
    rBIC5 = []
    for i in range(1,4):
        sigma1 = (np.linalg.norm(Y-result1[i-1][2]@X)**2)/(Y.shape[1])
        a1,b1,c1,d1,e1 = (i,rmax,rmax,rmax,rmax)
        df1 = 5*(a1+e1)+5*(a1*b1+b1*c1+c1*d1+d1*e1)+c1
        BIC1 = np.log(sigma1)+0.02*np.log(Y.shape[1])*(df1+1)/(Y.shape[1])
        rBIC1.append(BIC1)
        
        sigma2 = (np.linalg.norm(Y-result2[i-1][2]@X)**2)/(Y.shape[1])
        a2,b2,c2,d2,e2 = (rmax,i,rmax,rmax,rmax)
        df2 = 5*(a2+e2)+5*(a2*b2+b2*c2+c2*d2+d2*e2)+c2
        BIC2 = np.log(sigma2)+0.02*np.log(Y.shape[1])*(df2+1)/(Y.shape[1])
        rBIC2.append(BIC2) 
        
        sigma3 = (np.linalg.norm(Y-result3[i-1][2]@X)**2)/(Y.shape[1])
        a3,b3,c3,d3,e3 = (rmax,rmax,i,rmax,rmax)
        df3 = 5*(a3+e3)+5*(a3*b3+b3*c3+c3*d3+d3*e3)+c3
        BIC3 = np.log(sigma3)+0.02*np.log(Y.shape[1])*(df3+1)/(Y.shape[1])
        rBIC3.append(BIC3) 
        
        sigma4 = (np.linalg.norm(Y-result4[i-1][2]@X)**2)/(Y.shape[1])
        a4,b4,c4,d4,e4 = (rmax,rmax,rmax,i,rmax)
        df4 = 5*(a4+e4)+5*(a4*b4+b4*c4+c4*d4+d4*e4)+c4
        BIC4 = np.log(sigma4)+0.02*np.log(Y.shape[1])*(df4+1)/(Y.shape[1])
        rBIC4.append(BIC4) 
        
        sigma5 = (np.linalg.norm(Y-result5[i-1][2]@X)**2)/(Y.shape[1])
        a5,b5,c5,d5,e5 = (rmax,rmax,rmax,rmax,i)
        df5 = 5*(a5+e5)+5*(a5*b5+b5*c5+c5*d5+d5*e5)+c5
        BIC5 = np.log(sigma5)+0.02*np.log(Y.shape[1])*(df5+1)/(Y.shape[1])
        rBIC5.append(BIC5) 
        
    est_rank = [np.argsort(np.array(rBIC1))[0]+1]+[np.argsort(np.array(rBIC2))[0]+1]+[np.argsort(np.array(rBIC3))[0]+1]+[np.argsort(np.array(rBIC4))[0]+1]+[np.argsort(np.array(rBIC5))[0]+1]
    rBIC = rBIC1 + rBIC2 + rBIC3 +rBIC4 + rBIC5

    return (rBIC,est_rank,t1,t2)
    #return (X,Y,A_mat,A_ts)


In [ ]:
def d3(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    T=250
    p=125
    torch.random.seed()       
        
    #import time
    #start = time.time()
    global ts; global X; global Y;
    #ts = torch.empty((p, T), dtype=torch.float64)
    X = torch.empty((p, T), dtype=torch.float64)
    Y = torch.empty((p, T), dtype=torch.float64)
    X[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    #ts[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    for s in range(0,T):
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = X.numpy()
        Y = Y.numpy()
        if s!=T-1:
            X[:,s+1] = (np.transpose((Y[:,s].reshape(list(reversed(p_vec)),order="F")),[2,1,0])).reshape(p,order="F")
        #ts[:,s] = torch.tensor(A_mat) @ ts[:,s-1] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        #print(s)
    #end = time.time()
    #print(end-start)
    #ts = ts.nump()
    X = X.numpy()
    Y = Y.numpy()
    
    #a = list(range(1,4))
    #b = list(range(1,4))
    #c = list(range(1,4))
    #d = list(range(1,4))
    #e = list(range(1,4))
    #global paramlist;
    #paramlist = list(itertools.product(a,b,c,d,e))
    
    rmax = 3
    result1 = []
    result2 = []
    result3 = []
    result4 = []
    result5 = []
    t1 = 0
    t2 = 0
    for i in range(1,4):
        result1a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[i]+[rmax,rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result1.append(result1a)
        if result1a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result1a[1])>1998:
            t2 = t2 + 1
        
        result2a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax]+[i]+[rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result2.append(result2a)
        if result2a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result2a[1])>1998:
            t2 = t2 + 1
        
        result3a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax]+[i]+[rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result3.append(result3a)
        if result3a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result3a[1])>1998:
            t2 = t2 + 1
        
        result4a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax]+[i]+[rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result4.append(result4a)
        if result4a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result4a[1])>1998:
            t2 = t2 + 1
        
        result5a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax,rmax]+[i], eta=0.01, max_iter=2000, tol=1e-4)
        result5.append(result5a)
        if result5a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result5a[1])>1998:
            t2 = t2 + 1
        
        
    rBIC1 = []
    rBIC2 = []
    rBIC3 = []
    rBIC4 = []
    rBIC5 = []
    for i in range(1,4):
        sigma1 = (np.linalg.norm(Y-result1[i-1][2]@X)**2)/(Y.shape[1])
        a1,b1,c1,d1,e1 = (i,rmax,rmax,rmax,rmax)
        df1 = 5*(a1+e1)+5*(a1*b1+b1*c1+c1*d1+d1*e1)+c1
        BIC1 = np.log(sigma1)+0.02*np.log(Y.shape[1])*(df1+1)/(Y.shape[1])
        rBIC1.append(BIC1)
        
        sigma2 = (np.linalg.norm(Y-result2[i-1][2]@X)**2)/(Y.shape[1])
        a2,b2,c2,d2,e2 = (rmax,i,rmax,rmax,rmax)
        df2 = 5*(a2+e2)+5*(a2*b2+b2*c2+c2*d2+d2*e2)+c2
        BIC2 = np.log(sigma2)+0.02*np.log(Y.shape[1])*(df2+1)/(Y.shape[1])
        rBIC2.append(BIC2) 
        
        sigma3 = (np.linalg.norm(Y-result3[i-1][2]@X)**2)/(Y.shape[1])
        a3,b3,c3,d3,e3 = (rmax,rmax,i,rmax,rmax)
        df3 = 5*(a3+e3)+5*(a3*b3+b3*c3+c3*d3+d3*e3)+c3
        BIC3 = np.log(sigma3)+0.02*np.log(Y.shape[1])*(df3+1)/(Y.shape[1])
        rBIC3.append(BIC3) 
        
        sigma4 = (np.linalg.norm(Y-result4[i-1][2]@X)**2)/(Y.shape[1])
        a4,b4,c4,d4,e4 = (rmax,rmax,rmax,i,rmax)
        df4 = 5*(a4+e4)+5*(a4*b4+b4*c4+c4*d4+d4*e4)+c4
        BIC4 = np.log(sigma4)+0.02*np.log(Y.shape[1])*(df4+1)/(Y.shape[1])
        rBIC4.append(BIC4) 
        
        sigma5 = (np.linalg.norm(Y-result5[i-1][2]@X)**2)/(Y.shape[1])
        a5,b5,c5,d5,e5 = (rmax,rmax,rmax,rmax,i)
        df5 = 5*(a5+e5)+5*(a5*b5+b5*c5+c5*d5+d5*e5)+c5
        BIC5 = np.log(sigma5)+0.02*np.log(Y.shape[1])*(df5+1)/(Y.shape[1])
        rBIC5.append(BIC5) 
        
    est_rank = [np.argsort(np.array(rBIC1))[0]+1]+[np.argsort(np.array(rBIC2))[0]+1]+[np.argsort(np.array(rBIC3))[0]+1]+[np.argsort(np.array(rBIC4))[0]+1]+[np.argsort(np.array(rBIC5))[0]+1]
    rBIC = rBIC1 + rBIC2 + rBIC3 +rBIC4 + rBIC5

    return (rBIC,est_rank,t1,t2)
    #return (X,Y,A_mat,A_ts)


In [ ]:
def d4(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    T=350
    p = 125
    torch.random.seed()       
        
    #import time
    #start = time.time()
    global ts; global X; global Y;
    #ts = torch.empty((p, T), dtype=torch.float64)
    X = torch.empty((p, T), dtype=torch.float64)
    Y = torch.empty((p, T), dtype=torch.float64)
    X[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    #ts[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    for s in range(0,T):
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = X.numpy()
        Y = Y.numpy()
        if s!=T-1:
            X[:,s+1] = (np.transpose((Y[:,s].reshape(list(reversed(p_vec)),order="F")),[2,1,0])).reshape(p,order="F")
        #ts[:,s] = torch.tensor(A_mat) @ ts[:,s-1] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        #print(s)
    #end = time.time()
    #print(end-start)
    #ts = ts.nump()
    X = X.numpy()
    Y = Y.numpy()
    
    #a = list(range(1,4))
    #b = list(range(1,4))
    #c = list(range(1,4))
    #d = list(range(1,4))
    #e = list(range(1,4))
    #global paramlist;
    #paramlist = list(itertools.product(a,b,c,d,e))
    
    rmax = 3
    result1 = []
    result2 = []
    result3 = []
    result4 = []
    result5 = []
    t1 = 0
    t2 = 0
    for i in range(1,4):
        result1a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[i]+[rmax,rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result1.append(result1a)
        if result1a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result1a[1])>1998:
            t2 = t2 + 1
        
        result2a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax]+[i]+[rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result2.append(result2a)
        if result2a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result2a[1])>1998:
            t2 = t2 + 1
        
        result3a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax]+[i]+[rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result3.append(result3a)
        if result3a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result3a[1])>1998:
            t2 = t2 + 1
        
        result4a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax]+[i]+[rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result4.append(result4a)
        if result4a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result4a[1])>1998:
            t2 = t2 + 1
        
        result5a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax,rmax]+[i], eta=0.01, max_iter=2000, tol=1e-4)
        result5.append(result5a)
        if result5a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result5a[1])>1998:
            t2 = t2 + 1
        
        
    rBIC1 = []
    rBIC2 = []
    rBIC3 = []
    rBIC4 = []
    rBIC5 = []
    for i in range(1,4):
        sigma1 = (np.linalg.norm(Y-result1[i-1][2]@X)**2)/(Y.shape[1])
        a1,b1,c1,d1,e1 = (i,rmax,rmax,rmax,rmax)
        df1 = 5*(a1+e1)+5*(a1*b1+b1*c1+c1*d1+d1*e1)+c1
        BIC1 = np.log(sigma1)+0.02*np.log(Y.shape[1])*(df1+1)/(Y.shape[1])
        rBIC1.append(BIC1)
        
        sigma2 = (np.linalg.norm(Y-result2[i-1][2]@X)**2)/(Y.shape[1])
        a2,b2,c2,d2,e2 = (rmax,i,rmax,rmax,rmax)
        df2 = 5*(a2+e2)+5*(a2*b2+b2*c2+c2*d2+d2*e2)+c2
        BIC2 = np.log(sigma2)+0.02*np.log(Y.shape[1])*(df2+1)/(Y.shape[1])
        rBIC2.append(BIC2) 
        
        sigma3 = (np.linalg.norm(Y-result3[i-1][2]@X)**2)/(Y.shape[1])
        a3,b3,c3,d3,e3 = (rmax,rmax,i,rmax,rmax)
        df3 = 5*(a3+e3)+5*(a3*b3+b3*c3+c3*d3+d3*e3)+c3
        BIC3 = np.log(sigma3)+0.02*np.log(Y.shape[1])*(df3+1)/(Y.shape[1])
        rBIC3.append(BIC3) 
        
        sigma4 = (np.linalg.norm(Y-result4[i-1][2]@X)**2)/(Y.shape[1])
        a4,b4,c4,d4,e4 = (rmax,rmax,rmax,i,rmax)
        df4 = 5*(a4+e4)+5*(a4*b4+b4*c4+c4*d4+d4*e4)+c4
        BIC4 = np.log(sigma4)+0.02*np.log(Y.shape[1])*(df4+1)/(Y.shape[1])
        rBIC4.append(BIC4) 
        
        sigma5 = (np.linalg.norm(Y-result5[i-1][2]@X)**2)/(Y.shape[1])
        a5,b5,c5,d5,e5 = (rmax,rmax,rmax,rmax,i)
        df5 = 5*(a5+e5)+5*(a5*b5+b5*c5+c5*d5+d5*e5)+c5
        BIC5 = np.log(sigma5)+0.02*np.log(Y.shape[1])*(df5+1)/(Y.shape[1])
        rBIC5.append(BIC5) 
        
    est_rank = [np.argsort(np.array(rBIC1))[0]+1]+[np.argsort(np.array(rBIC2))[0]+1]+[np.argsort(np.array(rBIC3))[0]+1]+[np.argsort(np.array(rBIC4))[0]+1]+[np.argsort(np.array(rBIC5))[0]+1]
    rBIC = rBIC1 + rBIC2 + rBIC3 +rBIC4 + rBIC5

    return (rBIC,est_rank,t1,t2)
    #return (X,Y,A_mat,A_ts)


In [ ]:
def d5(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    T=450
    p = 125
    torch.random.seed()       
        
    #import time
    #start = time.time()
    global ts; global X; global Y;
    #ts = torch.empty((p, T), dtype=torch.float64)
    X = torch.empty((p, T), dtype=torch.float64)
    Y = torch.empty((p, T), dtype=torch.float64)
    X[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    #ts[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    for s in range(0,T):
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = X.numpy()
        Y = Y.numpy()
        if s!=T-1:
            X[:,s+1] = (np.transpose((Y[:,s].reshape(list(reversed(p_vec)),order="F")),[2,1,0])).reshape(p,order="F")
        #ts[:,s] = torch.tensor(A_mat) @ ts[:,s-1] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        #print(s)
    #end = time.time()
    #print(end-start)
    #ts = ts.nump()
    X = X.numpy()
    Y = Y.numpy()
    
    #a = list(range(1,4))
    #b = list(range(1,4))
    #c = list(range(1,4))
    #d = list(range(1,4))
    #e = list(range(1,4))
    #global paramlist;
    #paramlist = list(itertools.product(a,b,c,d,e))
    
    rmax = 3
    result1 = []
    result2 = []
    result3 = []
    result4 = []
    result5 = []
    t1 = 0
    t2 = 0
    for i in range(1,4):
        result1a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[i]+[rmax,rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result1.append(result1a)
        if result1a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result1a[1])>1998:
            t2 = t2 + 1
        
        result2a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax]+[i]+[rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result2.append(result2a)
        if result2a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result2a[1])>1998:
            t2 = t2 + 1
        
        result3a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax]+[i]+[rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result3.append(result3a)
        if result3a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result3a[1])>1998:
            t2 = t2 + 1
        
        result4a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax]+[i]+[rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result4.append(result4a)
        if result4a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result4a[1])>1998:
            t2 = t2 + 1
        
        result5a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax,rmax]+[i], eta=0.01, max_iter=2000, tol=1e-4)
        result5.append(result5a)
        if result5a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result5a[1])>1998:
            t2 = t2 + 1
        
        
    rBIC1 = []
    rBIC2 = []
    rBIC3 = []
    rBIC4 = []
    rBIC5 = []
    for i in range(1,4):
        sigma1 = (np.linalg.norm(Y-result1[i-1][2]@X)**2)/(Y.shape[1])
        a1,b1,c1,d1,e1 = (i,rmax,rmax,rmax,rmax)
        df1 = 5*(a1+e1)+5*(a1*b1+b1*c1+c1*d1+d1*e1)+c1
        BIC1 = np.log(sigma1)+0.02*np.log(Y.shape[1])*(df1+1)/(Y.shape[1])
        rBIC1.append(BIC1)
        
        sigma2 = (np.linalg.norm(Y-result2[i-1][2]@X)**2)/(Y.shape[1])
        a2,b2,c2,d2,e2 = (rmax,i,rmax,rmax,rmax)
        df2 = 5*(a2+e2)+5*(a2*b2+b2*c2+c2*d2+d2*e2)+c2
        BIC2 = np.log(sigma2)+0.02*np.log(Y.shape[1])*(df2+1)/(Y.shape[1])
        rBIC2.append(BIC2) 
        
        sigma3 = (np.linalg.norm(Y-result3[i-1][2]@X)**2)/(Y.shape[1])
        a3,b3,c3,d3,e3 = (rmax,rmax,i,rmax,rmax)
        df3 = 5*(a3+e3)+5*(a3*b3+b3*c3+c3*d3+d3*e3)+c3
        BIC3 = np.log(sigma3)+0.02*np.log(Y.shape[1])*(df3+1)/(Y.shape[1])
        rBIC3.append(BIC3) 
        
        sigma4 = (np.linalg.norm(Y-result4[i-1][2]@X)**2)/(Y.shape[1])
        a4,b4,c4,d4,e4 = (rmax,rmax,rmax,i,rmax)
        df4 = 5*(a4+e4)+5*(a4*b4+b4*c4+c4*d4+d4*e4)+c4
        BIC4 = np.log(sigma4)+0.02*np.log(Y.shape[1])*(df4+1)/(Y.shape[1])
        rBIC4.append(BIC4) 
        
        sigma5 = (np.linalg.norm(Y-result5[i-1][2]@X)**2)/(Y.shape[1])
        a5,b5,c5,d5,e5 = (rmax,rmax,rmax,rmax,i)
        df5 = 5*(a5+e5)+5*(a5*b5+b5*c5+c5*d5+d5*e5)+c5
        BIC5 = np.log(sigma5)+0.02*np.log(Y.shape[1])*(df5+1)/(Y.shape[1])
        rBIC5.append(BIC5) 
        
    est_rank = [np.argsort(np.array(rBIC1))[0]+1]+[np.argsort(np.array(rBIC2))[0]+1]+[np.argsort(np.array(rBIC3))[0]+1]+[np.argsort(np.array(rBIC4))[0]+1]+[np.argsort(np.array(rBIC5))[0]+1]
    rBIC = rBIC1 + rBIC2 + rBIC3 +rBIC4 + rBIC5

    return (rBIC,est_rank,t1,t2)
    #return (X,Y,A_mat,A_ts)


In [ ]:
def d6(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    T=550
    p = 125
    torch.random.seed()       
        
    #import time
    #start = time.time()
    global ts; global X; global Y;
    #ts = torch.empty((p, T), dtype=torch.float64)
    X = torch.empty((p, T), dtype=torch.float64)
    Y = torch.empty((p, T), dtype=torch.float64)
    X[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    #ts[:,0] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
    for s in range(0,T):
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = X.numpy()
        Y = Y.numpy()
        if s!=T-1:
            X[:,s+1] = (np.transpose((Y[:,s].reshape(list(reversed(p_vec)),order="F")),[2,1,0])).reshape(p,order="F")
        #ts[:,s] = torch.tensor(A_mat) @ ts[:,s-1] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        #print(s)
    #end = time.time()
    #print(end-start)
    #ts = ts.nump()
    X = X.numpy()
    Y = Y.numpy()
    
    #a = list(range(1,4))
    #b = list(range(1,4))
    #c = list(range(1,4))
    #d = list(range(1,4))
    #e = list(range(1,4))
    #global paramlist;
    #paramlist = list(itertools.product(a,b,c,d,e))
    
    rmax = 3
    result1 = []
    result2 = []
    result3 = []
    result4 = []
    result5 = []
    t1 = 0
    t2 = 0
    for i in range(1,4):
        result1a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[i]+[rmax,rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result1.append(result1a)
        if result1a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result1a[1])>1998:
            t2 = t2 + 1
        
        result2a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax]+[i]+[rmax,rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result2.append(result2a)
        if result2a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result2a[1])>1998:
            t2 = t2 + 1
        
        result3a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax]+[i]+[rmax,rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result3.append(result3a)
        if result3a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result3a[1])>1998:
            t2 = t2 + 1
        
        result4a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax]+[i]+[rmax], eta=0.01, max_iter=2000, tol=1e-4)
        result4.append(result4a)
        if result4a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result4a[1])>1998:
            t2 = t2 + 1
        
        result5a = PGD(Y, X, trueA=A_mat, p_vec=p_vec, mtl_rank=[rmax,rmax,rmax,rmax]+[i], eta=0.01, max_iter=2000, tol=1e-4)
        result5.append(result5a)
        if result5a[0][-1]>0.01:
            t1 = t1 + 1
        if len(result5a[1])>1998:
            t2 = t2 + 1
        
        
    rBIC1 = []
    rBIC2 = []
    rBIC3 = []
    rBIC4 = []
    rBIC5 = []
    for i in range(1,4):
        sigma1 = (np.linalg.norm(Y-result1[i-1][2]@X)**2)/(Y.shape[1])
        a1,b1,c1,d1,e1 = (i,rmax,rmax,rmax,rmax)
        df1 = 5*(a1+e1)+5*(a1*b1+b1*c1+c1*d1+d1*e1)+c1
        BIC1 = np.log(sigma1)+0.02*np.log(Y.shape[1])*(df1+1)/(Y.shape[1])
        rBIC1.append(BIC1)
        
        sigma2 = (np.linalg.norm(Y-result2[i-1][2]@X)**2)/(Y.shape[1])
        a2,b2,c2,d2,e2 = (rmax,i,rmax,rmax,rmax)
        df2 = 5*(a2+e2)+5*(a2*b2+b2*c2+c2*d2+d2*e2)+c2
        BIC2 = np.log(sigma2)+0.02*np.log(Y.shape[1])*(df2+1)/(Y.shape[1])
        rBIC2.append(BIC2) 
        
        sigma3 = (np.linalg.norm(Y-result3[i-1][2]@X)**2)/(Y.shape[1])
        a3,b3,c3,d3,e3 = (rmax,rmax,i,rmax,rmax)
        df3 = 5*(a3+e3)+5*(a3*b3+b3*c3+c3*d3+d3*e3)+c3
        BIC3 = np.log(sigma3)+0.02*np.log(Y.shape[1])*(df3+1)/(Y.shape[1])
        rBIC3.append(BIC3) 
        
        sigma4 = (np.linalg.norm(Y-result4[i-1][2]@X)**2)/(Y.shape[1])
        a4,b4,c4,d4,e4 = (rmax,rmax,rmax,i,rmax)
        df4 = 5*(a4+e4)+5*(a4*b4+b4*c4+c4*d4+d4*e4)+c4
        BIC4 = np.log(sigma4)+0.02*np.log(Y.shape[1])*(df4+1)/(Y.shape[1])
        rBIC4.append(BIC4) 
        
        sigma5 = (np.linalg.norm(Y-result5[i-1][2]@X)**2)/(Y.shape[1])
        a5,b5,c5,d5,e5 = (rmax,rmax,rmax,rmax,i)
        df5 = 5*(a5+e5)+5*(a5*b5+b5*c5+c5*d5+d5*e5)+c5
        BIC5 = np.log(sigma5)+0.02*np.log(Y.shape[1])*(df5+1)/(Y.shape[1])
        rBIC5.append(BIC5) 
        
    est_rank = [np.argsort(np.array(rBIC1))[0]+1]+[np.argsort(np.array(rBIC2))[0]+1]+[np.argsort(np.array(rBIC3))[0]+1]+[np.argsort(np.array(rBIC4))[0]+1]+[np.argsort(np.array(rBIC5))[0]+1]
    rBIC = rBIC1 + rBIC2 + rBIC3 +rBIC4 + rBIC5

    return (rBIC,est_rank,t1,t2)
    #return (X,Y,A_mat,A_ts)


In [ ]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=470))
        with open('33seprselectT50p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=470))
        with open('33seprselectT150p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=470))
        with open('33seprselectT250p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=470))
        with open('33seprselectT350p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d5,l),total=470))
        with open('33seprselectT450p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d6,l),total=500))
        with open('33seprselectT550p5R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
p_vec=[5,5,5]
mtl_rank = [1,2,2,2,2,2,1]
import pickle
with open('/home/r11user2/Documents/YF/TT_decomposition/seprank_selection/sigma42/42sepAp5r2.pkl','rb') as f:
    A_mat, A_ts, _ = pickle.load(f)

In [ ]:
lambda1=np.linalg.svd(A_ts.reshape(5,25*125,order="F"))[1][:2]
lambda2=np.linalg.svd(A_ts.reshape(25,5*125,order="F"))[1][:2]
lambda3=np.linalg.svd(A_ts.reshape(125,125,order="F"))[1][:2]
lambda4=np.linalg.svd(A_ts.reshape(125*5,25,order="F"))[1][:2]
lambda5=np.linalg.svd(A_ts.reshape(125*25,5,order="F"))[1][:2]
lambdatotal=np.concatenate((lambda1,lambda2,lambda3,lambda4,lambda5))
np.min(lambdatotal)

In [ ]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=470))
        with open('42seprselectT50p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=470))
        with open('42seprselectT150p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=470))
        with open('42seprselectT250p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=470))
        with open('42seprselectT350p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(470))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d5,l),total=470))
        with open('42seprselectT450p5R2part2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(500))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d6,l),total=500))
        with open('42seprselectT550p5R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
p_vec=[5,5,5]
p = np.prod(p_vec)
mtl_rank = [1,2,2,2,2,2,1]

In [ ]:
global stationary
p_vec=[5,5,5]
p = np.prod(p_vec)
mtl_rank = [1,2,2,2,2,2,1]
stationary = False
while not stationary:
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec[2],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[1])).reshape(p_vec[1]*mtl_rank[1],p_vec[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec[1],mtl_rank[2],order="F")
    G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[2])).reshape(p_vec[0]*mtl_rank[2],p_vec[0]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    sigma = np.diag([1.5,1.5])
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[4])).reshape(p_vec[0]*mtl_rank[4],p_vec[0]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[0],mtl_rank[4],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[5])).reshape(p_vec[1]*mtl_rank[5],p_vec[1]*mtl_rank[5]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[1],mtl_rank[5],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[5],:].reshape(mtl_rank[5],p_vec[2],mtl_rank[6],order="F")
    A_ts = core_to_tensor([G1,G2,G3,sigma,G4,G5,G6], list(reversed(p_vec))+p_vec)
    A_mat = A_ts.reshape(p, p, order="F")
    A_ts1 = np.transpose(A_ts,[2,1,0,3,4,5])
    stationary = np.max(np.abs(np.linalg.eig(A_mat)[0])) < 1 and np.max(np.abs(np.linalg.eig(A_ts1.reshape(p,p,order="F"))[0])) < 1
  

In [ ]:
A_ts = res[1]
lambda1=np.linalg.svd(A_ts.reshape(5,25*125,order="F"))[1][:2]
lambda2=np.linalg.svd(A_ts.reshape(25,5*125,order="F"))[1][:2]
lambda3=np.linalg.svd(A_ts.reshape(125,125,order="F"))[1][:2]
lambda4=np.linalg.svd(A_ts.reshape(125*5,25,order="F"))[1][:2]
lambda5=np.linalg.svd(A_ts.reshape(125*25,5,order="F"))[1][:2]
lambdatotal=np.concatenate((lambda1,lambda2,lambda3,lambda4,lambda5))
np.min(lambdatotal)

In [ ]:
np.linalg.norm(A_mat,"f")**2

In [ ]:
import pickle
with open('1515sepAp5r2.pkl','wb') as f:
    pickle.dump([A_mat, A_ts, A_ts1], f)

In [ ]:
import pickle
with open('1515sepAp5r2.pkl','rb') as f:
    res = pickle.load(f)
A_mat = res[0]

In [ ]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=300))
        with open('11seprselectT50p5R2part1.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=300))
        with open('11seprselectT150p5R2part1.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=300))
        with open('11seprselectT250p5R2part1.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=300))
        with open('11seprselectT350p5R2part1.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d5,l),total=300))
        with open('11seprselectT450p5R2part1.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
import pickle

with open('seprselectT1000p5R2.pkl','rb') as f:
    result = pickle.load(f)

In [ ]:
len(result)

In [ ]:
result

In [ ]:
list1 = []
a1 = 0
t1 = []
a2 = 0
t2 = []
a3 = 0
for i in range(len(result)):
    list1.append(result[i][1])
    if result[i][1]==[2,2,2,2,2]:
        a1 = a1 + 1
    t1.append(result[i][2])
    if result[i][2]>0:
        a2 = a2 + 1
    t2.append(result[i][3])
    if result[i][3]>0:
        a3 = a3 + 1

In [ ]:
a1

In [ ]:
a2

In [ ]:
list1